In [1]:
%load_ext autoreload
%autoreload 2

In [56]:
import pandas as pd
import pickle

from src.paths import MODEL_DIR, PREPROCESSED_DATA_DIR, RAW_DATA_DIR, SUBMISSION_DIR

In [5]:
with open(MODEL_DIR / 'model_1.pkl', 'rb') as file:
    model = pickle.load(file)

In [38]:
data_test = pd.read_pickle(PREPROCESSED_DATA_DIR / 'data_test.pkl')
data_test.head()

,Species,Latitude,Longitude,Dayofyear,Week,Tmax_1,DewPoint_1,WetBulb_1,PrecipTotal_1,Tmax_2,DewPoint_1_mean_l1_w4,PrecipTotal_2_mean_l7_w1,Tmin_1_mean_l10_w1,DewPoint_1_mean_l10_w1,Tmax_2_mean_l13_w10,Id
0,1.0,41.954690,-87.800991,183,27,83,51,60.0,0.0,84,57.5,0.0,61.0,52.0,80.4,3634
1,2.0,41.954690,-87.800991,183,27,83,51,60.0,0.0,84,57.5,0.0,61.0,52.0,80.4,3635
2,0.0,41.954690,-87.800991,183,27,83,51,60.0,0.0,84,57.5,0.0,61.0,52.0,80.4,3636
3,1.0,41.974089,-87.824812,183,27,83,51,60.0,0.0,84,57.5,0.0,61.0,52.0,80.4,3650
4,2.0,41.974089,-87.824812,183,27,83,51,60.0,0.0,84,57.5,0.0,61.0,52.0,80.4,3651


In [39]:
predictions = model.predict_proba(data_test.drop(['Id'], axis=1))[:,1]
data_test['proba'] = predictions
data_test.head()

,Species,Latitude,Longitude,Dayofyear,Week,Tmax_1,DewPoint_1,WetBulb_1,PrecipTotal_1,Tmax_2,DewPoint_1_mean_l1_w4,PrecipTotal_2_mean_l7_w1,Tmin_1_mean_l10_w1,DewPoint_1_mean_l10_w1,Tmax_2_mean_l13_w10,Id,proba
0,1.0,41.954690,-87.800991,183,27,83,51,60.0,0.0,84,57.5,0.0,61.0,52.0,80.4,3634,0.040453
1,2.0,41.954690,-87.800991,183,27,83,51,60.0,0.0,84,57.5,0.0,61.0,52.0,80.4,3635,0.035708
2,0.0,41.954690,-87.800991,183,27,83,51,60.0,0.0,84,57.5,0.0,61.0,52.0,80.4,3636,0.038719
3,1.0,41.974089,-87.824812,183,27,83,51,60.0,0.0,84,57.5,0.0,61.0,52.0,80.4,3650,0.028353
4,2.0,41.974089,-87.824812,183,27,83,51,60.0,0.0,84,57.5,0.0,61.0,52.0,80.4,3651,0.024990


In [46]:
sample = pd.read_csv(RAW_DATA_DIR / 'sampleSubmission.csv')
predictions = data_test[['Id', 'proba']]
predictions
results = pd.merge(sample, predictions, on='Id', how='outer').fillna(0)
results['WnvPresent']=results['proba']
results.drop('proba', axis=1, inplace=True)
results

,Id,WnvPresent
0,1,0.0
1,2,0.0
2,3,0.0
3,4,0.0
4,5,0.0
...,...,...
116288,116289,0.0
116289,116290,0.0
116290,116291,0.0
116291,116292,0.0


In [62]:
results.to_csv(SUBMISSION_DIR / 'test_results.csv', index=False)

In [63]:
!kaggle competitions submit -c predict-west-nile-virus -f /home/aga/repos/west_nile_virus/data/submission/test_results.csv -m test

100%|██████████████████████████████████████| 1.27M/1.27M [00:01<00:00, 1.08MB/s]
Successfully submitted to West Nile Virus Prediction

In [65]:
!kaggle competitions submissions -c predict-west-nile-virus

fileName                  date                 description  status    publicScore  privateScore  
------------------------  -------------------  -----------  --------  -----------  ------------  
test_results.csv          2024-08-31 13:31:14  test         complete  0.76447      0.73423       
test_results.csv          2024-08-31 13:28:20  test         error                                
xgb_predictions5.csv      2023-12-01 16:09:15               complete  0.75015      0.73316       
xgb_predictions4.csv      2023-12-01 11:14:34               complete  0.77961      0.75387       
xgb_predictions3 (1).csv  2023-12-01 10:54:35               complete  0.77779      0.74916       
xgb_predictions3.csv      2023-12-01 10:40:46               complete  0.64936      0.62700       
xgb_predictions2 (1).csv  2023-11-20 16:44:29               complete  0.71806      0.71191       
xgb_predictions2.csv      2023-11-20 16:24:28               complete  0.47421      0.47489       
xgb_predictions2.csv